# РИА-Новости

In [1]:
import requests
import numpy as np
import datetime
import re
from bs4 import BeautifulSoup

from fake_useragent import UserAgent

import time
import datetime
import pandas as pd
import pickle

from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
def Separator(vect, part):
    n = len(vect)
    vec_parts = [round(n/part)*i for i in range(part)]
    vec_parts.append(n)
    out = [vect[vec_parts[i]:vec_parts[i+1]] for i in range(part)]
    return(out)

def get_soup(url):
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    return soup

## 1. Download hrefs

In [6]:
url = 'https://ria.ru/services/archive/more.html?id=1572375708&date=20201025T140000'
news_information = [ ]

In [23]:
for i in range(10**6):
    
    soup = get_soup(url)
    url = 'https://ria.ru/' + soup.div.get('data-next-url')

    news = soup.find_all('div', {'class' : "list-item"})
    for item in news:
        inf = { }
        inf['href'] = item.a.get("href")
        try:
            predate = item.a.get("href").split('/')[-2]
            inf['date'] = predate[:4] + '-' + predate[4:6] + '-' + predate[6:]
        except:
            print('Проблемы с датой', inf['href'])
            inf['date'] = inf['href']
        inf['time'] = item.find("div", {"class" : "list-item__date"}).text
        inf['title'] = item.find("a", {"class": "list-item__title"}).text
        news_information.append(inf)
        
    if i % 500 == 0:    
        print(inf['date'], inf['time'])
        
    if inf['date'][:7] == '2020-05':
        break

2020-06-03 3 июня, 21:24
Проблемы с датой 02062020
Проблемы с датой 01062020


In [24]:
news_information[-1]

{'href': 'https://ria.ru/20200531/1572262866.html',
 'date': '2020-05-31',
 'time': '31 мая, 23:34',
 'title': 'Астронавт Бенкен рассказал об ощущениях от полета на Crew Dragon '}

In [30]:
df = pd.DataFrame(news_information)
print(df.shape)
df.drop_duplicates(subset = ['href','date'],inplace=True)
print(df.shape)

real_date = [ ]
years = {str(item) for item in range(2004,2021)}
base_date = df.date.iloc[0]
for item in df.date:
    if item[:4] in years:
        base_date = item
        real_date.append(item)
    else:
        real_date.append(base_date)

        
df.date = real_date
x = df.to_dict('records')
x[0]

(121920, 4)
(121658, 4)


{'href': 'https://ria.ru/20201025/serebro-1581440663.html',
 'date': '2020-10-25',
 'time': '13:36',
 'title': 'Фигура экс-солистки “Серебра” вызвала споры среди поклонников '}

In [33]:
!ls news_data/5.ria

lemm


In [34]:
# x_2020 = [item for item in x if item['date'][:4] == '2019']

with open('news_data/5.ria/titles_new', 'wb') as f:
    pickle.dump(x, f)
    
# x_2018 = [item for item in x if item['date'][:4] == '2018']

# with open('ria_titles_2018.pickle', 'wb') as f:
#     pickle.dump(x_2018, f)

## 2. Download news

Скрипт ниже с одного ip работает максимально в два потока, иначе сервер выбрасывает на с сайта. Из-за того, что мне было лень паралелить через тор всё это с разными ip, я просто поднял в яндекс-облаке 6 машин и на каждой качал свой батч в два потока. 

In [4]:
!ls news_data/5.ria

lemm       titles_new


In [3]:
with open('news_data/5.ria/titles_new', 'rb') as f:
    x_2020 = pickle.load(f)

In [4]:
len(x_2020)

121658

In [5]:
def get_ria_news(item_from_vect):
    url = item_from_vect['href']
    try:
        soup = get_soup(url)
        text = soup.find_all('div', {'class' : 'article__text'})

        item_from_vect['year'] = item_from_vect['date'].split('-')[0]
        item_from_vect['month'] = item_from_vect['date'].split('-')[1]
        item_from_vect['day'] = item_from_vect['date'].split('-')[2]

        item_from_vect['snippet'] = soup.find_all('meta',  {'name' : 'description'})[0].get('content')
        item_from_vect['text'] = ' '.join([tx.text for tx in text])

        # категория
        item_from_vect['category'] = soup.find_all('meta', {'name' : 'analytics:rubric'})[0].get('content')

        # полезная метаинформация 
        item_from_vect['keywords'] = soup.find_all('meta', {'name' : 'keywords'})[0].get('content')
        item_from_vect['tags'] = soup.find_all('meta',  {'name' : 'analytics:tags'})[0].get('content')

        # число показов страницы
        # item_from_vect['shows'] = soup.find_all('span', {"class" : "statistic__item m-views"})[0].text

        # хз пригодится ли, но ссылка на титульную картинку 
        item_from_vect['image'] = soup.find_all('meta',  {'property' : 'og:image'})[0].get('content')
        return item_from_vect
    
    except:
        
        print(url)
        return item_from_vect

In [6]:
get_ria_news(item_from_vect)

NameError: name 'item_from_vect' is not defined

In [13]:
i = 0

x_batch = Separator(x_2020, 500)

result = [ ]
for batch in tqdm(x_batch[(198 + 180):]):
    
    n_jobs = 2 # параллелим на все ядра
    result_cur = Parallel(n_jobs=n_jobs)(delayed(get_ria_news)(
        text) for text in batch)

    print('Скачал батч номер ' + str(i))
    i += 1
    result.extend(result_cur)

    clean = [itog for itog in result if len(itog.keys()) == 13]
    print('Сейчас у нас есть ' + str(len(clean)) +' новостей')

    bad = [itog for itog in result if len(itog.keys()) < 13]
    print('Сделано ' + str(len(bad)) +' ошибок')

  1%|          | 1/122 [04:13<8:31:27, 253.61s/it]

Скачал батч номер 0
Сейчас у нас есть 243 новостей
Сделано 0 ошибок


  2%|▏         | 2/122 [07:44<8:01:21, 240.68s/it]

Скачал батч номер 1
Сейчас у нас есть 486 новостей
Сделано 0 ошибок


  2%|▏         | 3/122 [12:33<8:26:27, 255.36s/it]

...

Скачал батч номер 119
Сейчас у нас есть 29147 новостей
Сделано 13 ошибок


 99%|█████████▉| 121/122 [6:10:51<03:01, 181.87s/it]

Скачал батч номер 120
Сейчас у нас есть 29390 новостей
Сделано 13 ошибок


100%|██████████| 122/122 [6:15:35<00:00, 184.72s/it]

Скачал батч номер 121
Сейчас у нас есть 29790 новостей
Сделано 14 ошибок


In [14]:
len(result)

29804

In [15]:
clean = [itog for itog in result if len(itog.keys()) == 14]
print('Сейчас у нас есть ' + str(len(clean)) +' новостей')

bad = [itog for itog in result if len(itog.keys()) < 14]
print('Сделано ' + str(len(bad)) +' ошибок')

Сейчас у нас есть 0 новостей
Сделано 29804 ошибок


In [16]:
!ls news_data/

1.rbc                      ria_news_new_part1.pickle
2.lenta                    ria_news_new_part2.pickle
3.interfax                 tass_news_new_part1.pickle
4.tass                     tass_news_new_part2.pickle
5.ria                      tass_titles_new.pickle
pickle_splitter.ipynb


In [17]:
with open('news_data/ria_news_new_part3.pickle', 'wb') as f:
    pickle.dump(result, f)